In [ ]:
from sympy import symbols, parsing, expand, simplify
from  numpy import poly1d, around
from Interpolation import *
from Helper_functions import *

In [ ]:
def piece_wise_poly(x_v, f_v, show_working=False):
    calc = ''
    for i in range(len(x_v)):
        t_x_v = x_v.copy()
        if len(t_x_v)>1:
            add = ''
            if i+1 != len(x_v): add='+'
            t_x_v.pop(i)
            ex1 = f'{getFactors(t_x_v, len(t_x_v))} / '
            ex2 = f'({getFactors(t_x_v, len(t_x_v), x_v[i])})  * ({f_v[i]}) {add} '
            calc += ex1 + ex2
            
            if show_working:
                print('\t' + ex1 + ' ' + ex2, end='')
    if show_working: print('\n        = ', end='')
    return sympy.expand(calc)

def Piecewise_Interpolation(x_list, f_values, gap, show_working=False):
    if gap > len(x_list)-1:
        print(f'[-] In Order to find the Polynomial of order {gap}, len(x_list) should be greater than gap')
        return None
    
    intervals = getIntervals(x_list, gap)
    f_intervals = getIntervals(f_values, gap)
    pwi = {1:'Linear', 2:'Quadratic', 3:'Cubic', 4:'Quartic'}
    print(f'By Piecewise {pwi[gap]} Interpolating Formula we have, ')
    
    for _ in range(len(intervals)):
        print("\tBy Lagrange's Interpolation Formula\n\n   f(x) = " , end='\n' if show_working else '')
        print(piece_wise_poly(intervals[_], f_intervals[_], show_working), end=' ')
        print(' x  ∈  [', ", ".join([str(intervals[_][0]), str(intervals[_][-1])]), ']')
        print('----'*15)

In [ ]:
def lgrange_Bivariate_I(x_list, alpha='x', write=False, str_format=True):
    result = []
    result_2 = []
    for i in range(len(x_list)):
        denon_str = ''
        num_str = ''
        mul_poly = poly1d([1])
        dr = 1
        for j in range(len(x_list)):
            if i!=j:
                num_str += f'({alpha} - {x_list[j]}) '
                mul_poly *= poly1d([1, -x_list[j]])
                denon_str += f'({x_list[i]} - {x_list[j]})'
                dr *= x_list[i] - x_list[j]
                
        res = mul_poly/dr
        result_2.append(f'{num_str.replace(") (", ") * (")}/ ({dr})')
        num_str = num_str.replace(") (", ') * (')
        
        if write:
            print(f'    {num_str}/({denon_str})')
            print('\t' + " ".join(coef_2_str(res.coef[::-1], alpha)))
            print('---------------------------------------------')
            
        result.append(res.coef[::-1])

    if str_format:
        return result_2
    
    return result


In [ ]:
def Lagrange_Bivariate_IF(x_vals, y_vals, arr, show_working=False):
    m, n = len(x_vals), len(y_vals)
    if show_working:
        print(f"Here m: {m}, n: {n}\n")
    
    x__ = lgrange_Bivariate_I(x_vals)
    y__ = lgrange_Bivariate_I(y_vals, 'y')
    
    if show_working:
        print(f"\nBy Lagrange's Bivariate Interpolating Formula \n\nP(x, y) = ", end='')
    for i in range(len(arr)):
        add4, tab = '', ''
        if i: tab = '\t'
        if i: add4 = ' + '
        if show_working:
            print(f'{tab}{add4}X_{m-1},{x_vals[i]} * ',end='[')
        for j in range(len(arr[i])):
            add = ''
            if j+1 != len(arr[i]):
                add = '+'
            if show_working:
                print(f'  Y_{n-1},{y_vals[j]} * (F_{x_vals[i]}_{y_vals[j]}) {add}', end=' ')
        if show_working:
            print(']')
    
    if show_working:
        print('\n= ', end='')
        
    t = ''
    for i in range(len(arr)):
        res_str = ''
        add4 = ''
        
        if i: add4 = ' + '
        if show_working:
            print(f'{add4}({x__[i]}) * ',end='[ ')
            
        res_str += f'({x__[i]}) * ('
        for j in range(len(arr[i])):
            add = ''
            if j+1 != len(arr[i]):
                add = ' + '
            add2 = ''
            if j: add2 = ' + '
            res_str += f'({y__[j]} * {arr[i][j]}) {add}'
            if show_working:
                print(f'({y__[j]} * {arr[i][j]}) {add}', end='')
            
        res_str += ')'
            
        add3 = ''
        if i+1 != len(arr):add3 = ' + '

        t += str(expand(res_str)) + add3
        if show_working:
            print(']')

    if show_working:
        print('\n ∴ P(x, y) = ' , simplify(t))
        
    return simplify(t)



In [ ]:
def d_fx_f(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, 1)
    return round(((f[i_][y_at] - f[x_at][y_at])/h), n)

def d_fx_b(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, -1)
    return round(((f[x_at][y_at] - f[i_][y_at])/h), n)

def d_fx_c(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, 1), value_at(x_vals, x_at, -1)
    return round(((f[i_[0]][y_at] - f[i_[1]][y_at])/(2*h)), n)

In [ ]:
def d_fy_f(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, 1)
    return round(((f[x_at][j_] - f[x_at][y_at])/k), n)

def d_fy_b(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, -1)
    return round(((f[x_at][y_at] - f[x_at][j_])/k), n)

def d_fy_c(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, 1), value_at(y_vals, y_at, -1)
    return round(((f[x_at][j_[0]] - f[x_at][j_[1]])/(2*k)), n)

In [ ]:
def d2_fx_f(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, 2), value_at(x_vals, x_at, 1)
    return round((f[i_[0]][y_at] - 2*f[i_[1]][y_at] + f[x_at][y_at])/h**2, n)

def d2_fx_b(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, -1), value_at(x_vals, x_at, -2)
    return round((f[x_at][y_at] - 2*f[i_[i_[0]]][y_at] + f[i_[1]][y_at])/h**2, n)

def d2_fx_c(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, -1), value_at(x_vals, x_at, 1)
    return round((f[i_[0]][y_at] - 2*f[x_at][y_at] + f[i_[1]][y_at])/h**2, n)

In [ ]:
def d2_fy_f(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, 2), value_at(y_vals, y_at, 1)
    return round((f[x_at][j_[0]] - 2*f[x_at][j_[1]] + f[x_at][y_at])/k**2, n)

def d2_fy_b(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, -1), value_at(y_vals, y_at, -2)
    return round((f[x_at][y_at] - 2*f[x_at][j_[0]] + f[x_at][j_[1]])/k**2, n)

def d2_fy_c(y_vals, x_at, y_at, f, a, n=5):
    j_ = value_at(y_vals, y_at, -1), value_at(y_vals, y_at, 1)
    return round((f[x_at][j_[0]] - 2*f[x_at][y_at] + f[x_at][j_[1]])/k**2, n)

In [ ]:
def d2_fxy(x_vals, y_vals, x_at, y_at, h, k, f, n=5):
    i_ = value_at(x_vals, x_at, 1), value_at(x_vals, x_at, -1)
    j_ = value_at(y_vals, y_at, 1), value_at(y_vals, y_at, -1)
    nr = f[i_[0]][j_[0]] - f[i_[0]][j_[1]] - f[i_[1]][j_[0]] + f[i_[1]][j_[1]]
    dnr = 4 * h * k
    return round(nr/dnr, n)

def d2_fyx(x_vals, y_vals, x_at, y_at, h, k, f, n=5):
    i_ = value_at(x_vals, x_at, 1), value_at(x_vals, x_at, -1)
    j_ = value_at(y_vals, y_at, 1), value_at(y_vals, y_at, -1)
    nr = f[i_[0]][j_[0]] - f[i_[0]][j_[1]] - f[i_[1]][j_[0]] + f[i_[1]][j_[1]]
    dnr = 4 * h * k
    return round(nr/dnr, n)

In [ ]:
def del_x_j(x_0, y_0, h, f):
    return check_d_n_r((f[x_0 + h][y_0] - f[x_0][y_0])/(2*h))

def del_y_j(x_0, y_0, k, f):
    return check_d_n_r((f[x_0][y_0+k] - f[x_0][y_0])/(2*k))

def del_x(x_0, y_0, h, f):
    if type(f)==dict:
        return f[x_0 + h][y_0] - f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, x_0+h, y_0) - sub_vals(f, x_0, y_0)
    
def del_y(x_0, y_0, k, f):
    if type(f)==dict:
        return f[x_0][y_0+k] - f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, x_0, y_0+k) - sub_vals(f, x_0, y_0)
    
def del_xx(x_0, y_0, h, f, n=5):
    c1, c2 = round(x_0 + 2* h, n), round(x_0+h, n)
    if type(f)==dict:
        return f[c1][y_0] - 2 * f[c2][y_0] + f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, c1, y_0) - 2 * sub_vals(f, c2, y_0) + sub_vals(f, x_0, y_0)

def del_yy(x_0, y_0, k, f, n=5):
    c1, c2 = round(y_0+2*k, n), round(y_0+k, n)
    if type(f)==dict:
        return f[x_0][c1] - 2 * f[x_0][c2] + f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, x_0, c1) - 2 * sub_vals(f, x_0, c2) + sub_vals(f, x_0, y_0)

def del_xy(x_0, y_0, h, k, f, n=5):
    c1, c2 = round(x_0+h, n), round(y_0+k, n)
    if type(f)==dict:
        return f[c1][c2] - f[c1][y_0] - f[x_0][c2] + f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, c1, c2) - sub_vals(f, c1, y_0) - sub_vals(f, x_0, c2) + sub_vals(f, x_0, y_0)

In [ ]:
def jacobian_matrix(x_vals, y_vals, f1, f2, x_0, y_0, print_table = False):
    if print_table:
        print_details(x_vals, y_vals, f1)
        print_details(x_vals, y_vals, f2)
    
    h, k = getHeight(x_vals), getHeight(y_vals)
    arr = make_table_xy(x_vals, y_vals, f1)
    dxf1 = del_x_j(x_0, y_0, h, arr)
    dyf1 = del_y_j(x_0, y_0, k, arr)

    arr = make_table_xy(x_vals, y_vals, f2)
    dxf2 = del_x_j(x_0, y_0, h, arr)
    dyf2 = del_y_j(x_0, y_0, k, arr)

    J_mat = [[dxf1, dyf1], [dxf2, dyf2]]
    
    if print_table:
        print(" Jacobian Matrix = ", end='')
        DrawMat(J_mat)
    
    return J_mat

In [ ]:
def Newton_Bivariate_interpolation(x_vals, y_vals, arr, at_x='x', at_y='y', show_working=False):
    if len(x_vals)>3 or len(y_vals)>3:
        print("\n  [-] This Funtion Can only handle x and y at 3 values ...  ")
        return None
    
    x_0, y_0 = x_vals[0], y_vals[0]
    h, k = getHeight(x_vals), getHeight(y_vals)
    f = get_x_y_at(x_vals, y_vals, arr)
    if show_working: 
        print(f'  Here h : {h}, k : {k}, x0 : {x_0}, y0 : {y_0}', end = ', ')
    t = ''
    for i in range(len(x_vals)):
        if i==0:
            t += f'{f[x_0][y_0]} + '

        else:
            if i == 1:
                fwd_val_x = del_x(x_0, y_0, h, f)
                fwd_val_y = del_y(x_0, y_0, k, f)
                if show_working: 
                    print(f'∆x = {fwd_val_x}, ∆y = {fwd_val_y}', end=', ')

            elif i==2:
                fwd_val_x = del_xx(x_0, y_0, h, f)
                fwd_val_y = del_yy(x_0, y_0, k, f)
                fwd_val_xy = del_xy(x_0, y_0, h, k, f)
                
                if show_working:
                    print(f'∆xx = {fwd_val_x}, ∆yy = {fwd_val_y}, ∆xy = {fwd_val_xy}')
                    
            s1 = getFactors(x_vals, i, at_x) +  ' * '
            s2 = ' + '+ getFactors(y_vals, i, at_y) + ' * '

            s1 += f'({fwd_val_x}) '
            s2 += f'({fwd_val_y}) '
            total = s1 
            if i==2:
                total += f" + 2/({h}*{k}) * {getFactors(x_vals, 1)} * {getFactors(y_vals, 1, 'y')} * ({fwd_val_xy})"
            total +=  s2
            mul = ''
            if i>1:
                mul = f'1/{fac(i)} * '

            add = ''
            if i+1 != len(x_vals): add = '+'
            t += f'{mul}({total}) {add} '
            
    if show_working:
        print('\n  f(x) =',t)
        
    return sympy.simplify(t)

## Index 

### 1. [Numerical Differentiation](#Numerical-Differentiation)
### 2. [Piecewise  Interpolation](#Piecewise-Interpolation)
### 3. [Lagrange's Bivariate Interpolation](#Lagranges-Bivariate-Interpolation)
### 4. [Newton's Bivariate Interpolation](#Newtons-Bivariate-Interpolation)
### 5. [Interpolation for Numerical Partial Differentiation](#Interpolation-for-Numerical-Partial-Differentiation)
### 6. [Jacobian Matrix](#Jacobian-Matrix)
### 7. [Forward Difference Value For Bivariate](#Forward-Difference-Value-For-Bivariate)

## Numerical Differentiation

In [ ]:
x_values, f_values = [0.4, 0.5, 0.6, 0.7, 0.8], [1.5836, 1.7974, 2.0442, 2.3275, 2.6511]
DrawForwardDiffTable(x_values, f_values, rn=5)
print()
diff_eq, h, a, x, at_x = Numerical_Diff(x_values, f_values)

n = len(" + ".join(diff_eq))
print('-'*(n-45))
print(f" ∴ f(a+xh) = {' + '.join(diff_eq)}")
print('-'*(n-45))

cal_derivative(diff_eq, h, x, at_x,  1) # 1st order derivative
cal_derivative(diff_eq, h, x, at_x, 2)  # 2nd order derivative

## Piecewise Interpolation

In [ ]:
x_list, f_values, gap = [2, 2.2, 2.6], [8, 9.24, 11.96], 2

Draw_table(x_list, f_values)
Piecewise_Interpolation(x_list,f_values, gap, True)

## Lagranges Bivariate Interpolation

In [ ]:
x_vals, y_vals = [0, 1, 2], [0, 1, 2]

In [ ]:
arr = get_f_x_y(x_vals, y_vals)

In [ ]:
create_xy_table(x_vals, y_vals, arr)

In [ ]:
t = Lagrange_Bivariate_IF(x_vals, y_vals, arr)
exp = simplify(t)

at_x, at_y = 1, 2

ans = expand(t).subs({'x':at_x, 'y':at_y})
print(f' ∴ P({at_x},{at_y}) = {exp} =  {ans}')

## Newtons Bivariate Interpolation

In [ ]:
x_vals, y_vals =  [1, 2], [2, 3]
arr = get_f_x_y(x_vals, y_vals)
create_xy_table(x_vals, y_vals, arr)
exp = Newton_Bivariate_interpolation(x_vals, y_vals, arr, show_working=True)
at_x, at_y = 2, 3
print(f"\n  ∴ P({at_x},{at_y}) = {exp} =  {exp.subs({'x':at_x, 'y':at_y})}")

## Interpolation for Numerical Partial Differentiation

In [ ]:
x_vals = [0.2, 0.3]
y_vals = [0.1, 0.2]

h, k = getHeight(x_vals), getHeight(y_vals)
x_at, y_at = float(input("At x: ")), float(input("At y: "))
print()

go = True
if h is None or k is None:
    print("[!] Check the intervals. !!!")
    go = False
    
if go:
    arr = get_f_x_y(x_vals, y_vals)
    f = get_x_y_at(x_vals, y_vals, arr)
    print(f"\n  df/dx = {d_fx_f(x_vals, x_at, y_at, h, f)}, df/dy = {d_fy_f(y_vals, x_at, y_at, k, f)} \
    at ({x_at}, {y_at})")